In [ ]:
import osmnx as ox
import networkx as nx
import folium as fl
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt



def haversine(a, b):
    (lat1 , lon1 )= a
    (lat2,  lon2 ) = b
    R = 6372.8  # Earth radius in kilometers
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R*c

def get_the_map(start_coordination, end_coordination, type):
    icons = {'CRIME':'star', 'FIRE':'fire-extinguisher', 'INJURY':'star-of-life'}
    colors = {'CRIME':'blue', 'FIRE':'red', 'INJURY':'pink'}
    locator = Nominatim(user_agent = "Khouribga Map")
    place     = 'Khouribga, Morocco'# find shortest route based on the mode of travel
    mode      = 'drive'
    optimizer = 'length'

    graph = ox.graph_from_place(place, network_type = mode)

    orig_node = ox.distance.nearest_nodes(graph, start_coordination[1],
                                      start_coordination[0])# find the nearest node to the end location
    dest_node = ox.distance.nearest_nodes(graph, end_coordination[1],
                                      end_coordination[0])

    def heuristic(node, goal):
        # Calculate the straight-line distance between the node and the goal
        node_data = graph.nodes[node]
        goal_data = graph.nodes[goal]
        node_coords = (node_data['y'], node_data['x'])
        goal_coords = (goal_data['y'], goal_data['x'])
        distance = haversine(node_coords, goal_coords)

        # Calculate the number of turns between the current node and the goal
        path = nx.shortest_path(graph, node, goal, weight='length')
        turns = sum(1 for i in range(1, len(path)-1) if graph.nodes.get(path[i], {}).get('lanes'))
        print('turns : ', turns)
        return distance + turns
    
    shortest_route = nx.astar_path(graph,
                                  orig_node,
                                  dest_node,
                                  heuristic=heuristic,
                                  weight="cost",
                                  )

    shortest_route_map = ox.plot_route_folium(graph, shortest_route,tiles='openstreetmap')
    fl.Marker(start_coordination, icon=fl.Icon(color=colors[type], icon=icons[type], prefix="fa")).add_to(shortest_route_map)
    fl.Marker(end_coordination, icon=fl.Icon(color="orange", icon="triangle-exclamation", prefix="fa")).add_to(shortest_route_map)
    return shortest_route_map
get_the_map([32.902222, -6.915341], [32.902492, -6.912423], "CRIME")

In [ ]:

def get_the_map(start_coordination, end_coordination, type):
    icons = {'CRIME':'star', 'FIRE':'fire-extinguisher', 'INJURY':'star-of-life'}
    colors = {'CRIME':'blue', 'FIRE':'red', 'INJURY':'pink'}
    locator = Nominatim(user_agent = "Khouribga Map")
    place     = 'Khouribga, Morocco'
    mode      = 'drive'
    optimizer = 'length'

    graph = ox.graph_from_place(place, network_type=mode)

    # Define the weight of each edge based on the number of turns required to traverse that edge and the length of the edge
    for u, v, k, data in graph.edges(keys=True, data=True):
        # data['weight'] = data['length'] + data['turns']
        print(data)

    orig_node = ox.distance.nearest_nodes(graph, start_coordination[1], start_coordination[0])
    dest_node = ox.distance.nearest_nodes(graph, end_coordination[1], end_coordination[0])

    # Find the shortest path between the start and end nodes based on the weight of the edges
    shortest_route = nx.shortest_path_weighted(graph, orig_node, dest_node, weight='weight')

    shortest_route_map = ox.plot_route_folium(graph, shortest_route, tiles='openstreetmap')
    fl.Marker(start_coordination, icon=fl.Icon(color=colors[type], icon=icons[type], prefix="fa")).add_to(shortest_route_map)
    fl.Marker(end_coordination, icon=fl.Icon(color="orange", icon="triangle-exclamation", prefix="fa")).add_to(shortest_route_map)
    return shortest_route_map


get_the_map([32.902492, -6.912423], [32.902492, -6.912423], "CRIME")

{'osmid': 591449881, 'oneway': False, 'lanes': '4', 'name': 'Boulevard Moulay Idriss 1', 'highway': 'secondary', 'maxspeed': '60', 'reversed': False, 'length': 46.707}
{'osmid': 156360440, 'oneway': True, 'lanes': '2', 'highway': 'secondary', 'maxspeed': '40', 'reversed': False, 'length': 35.638, 'geometry': <LINESTRING (-6.926 32.872, -6.926 32.872, -6.926 32.872)>}
{'osmid': 185019294, 'oneway': False, 'lanes': '4', 'ref': 'RN12', 'highway': 'primary', 'maxspeed': '80', 'reversed': False, 'length': 196.29200000000003, 'geometry': <LINESTRING (-6.914 32.899, -6.913 32.898, -6.912 32.898, -6.912 32.898)>}
{'osmid': 410642948, 'oneway': True, 'lanes': '1', 'ref': 'RN12', 'highway': 'primary', 'maxspeed': '60', 'reversed': False, 'length': 66.965, 'geometry': <LINESTRING (-6.914 32.899, -6.914 32.899, -6.914 32.899, -6.914 32.899)>}
{'osmid': 108226683, 'oneway': True, 'highway': 'secondary', 'reversed': False, 'length': 34.074, 'geometry': <LINESTRING (-6.933 32.901, -6.933 32.9, -6.933

AttributeError: module 'networkx' has no attribute 'shortest_path_weighted'